In [2]:
import pandas as pd 
from datetime import datetime, timedelta

In [13]:

data = pd.read_csv('ELIA_23August2024.csv',sep=';', index_col=0, parse_dates=True)

In [14]:
# find the first not na in Total Load 
first_valid_index = data['Total Load'].first_valid_index()
data = data.loc[data.index < first_valid_index]
data

Resolution code  Total Load  Most recent forecast  \
Datetime                                                                      
2024-08-23 04:15:00+02:00           PT15M     7378.27               7387.74   
2024-08-23 04:00:00+02:00           PT15M     7361.58               7382.33   
2024-08-23 03:45:00+02:00           PT15M     7215.68               7227.33   
2024-08-23 03:30:00+02:00           PT15M     7273.88               7218.03   
2024-08-23 03:15:00+02:00           PT15M     7407.23               7230.58   
...                                   ...         ...                   ...   
2015-01-01 01:00:00+01:00           PT15M     9755.00               9222.33   
2015-01-01 00:45:00+01:00           PT15M     9821.78               9025.46   
2015-01-01 00:30:00+01:00           PT15M     9952.87               9174.72   
2015-01-01 00:15:00+01:00           PT15M    10051.28               9329.17   
2015-01-01 00:00:00+01:00           PT15M    10142.19               9496.05   

                           Most recent P10  Most recent P90  \
Datetime                                                      
2024-08-23 04:15:00+02:00          7206.74          7568.74   
2024-08-23 04:00:00+02:00          7201.47          7563.20   
2024-08-23 03:45:00+02:00          7046.09          7408.57   
2024-08-23 03:30:00+02:00          7037.02          7399.03   
2024-08-23 03:15:00+02:00          7049.28          7411.88   
...                                    ...              ...   
2015-01-01 01:00:00+01:00          8880.23          9564.44   
2015-01-01 00:45:00+01:00          8432.07          9618.85   
2015-01-01 00:30:00+01:00          8571.51          9777.92   
2015-01-01 00:15:00+01:00          8715.80          9942.53   
2015-01-01 00:00:00+01:00          8871.71         10120.38   

                           Day-ahead 6PM forecast  Day-ahead 6PM P10  \
Datetime                                                               
2024-08-23 04:15:00+02:00                 7530.12            7309.20   
2024-08-23 04:00:00+02:00                 7502.57            7282.47   
2024-08-23 03:45:00+02:00                 7485.72            7264.91   
2024-08-23 03:30:00+02:00                 7478.08            7257.50   
2024-08-23 03:15:00+02:00                 7486.94            7266.10   
...                                           ...                ...   
2015-01-01 01:00:00+01:00                10130.36            9777.87   
2015-01-01 00:45:00+01:00                 9216.09            8625.97   
2015-01-01 00:30:00+01:00                 9367.66            8767.84   
2015-01-01 00:15:00+01:00                 9526.54            8916.55   
2015-01-01 00:00:00+01:00                 9697.59            9076.64   

                           Day-ahead 6PM P90  Week-ahead forecast  
Datetime                                                           
2024-08-23 04:15:00+02:00            7751.03                  NaN  
2024-08-23 04:00:00+02:00            7722.68                  NaN  
2024-08-23 03:45:00+02:00            7706.53                  NaN  
2024-08-23 03:30:00+02:00            7698.66                  NaN  
2024-08-23 03:15:00+02:00            7707.79                  NaN  
...                                      ...                  ...  
2015-01-01 01:00:00+01:00           10482.86              9203.25  
2015-01-01 00:45:00+01:00            9806.20              9319.45  
2015-01-01 00:30:00+01:00            9967.47              9473.22  
2015-01-01 00:15:00+01:00           10136.53              9653.31  
2015-01-01 00:00:00+01:00           10318.53              9833.37  

[338126 rows x 9 columns]

In [15]:
data = data.fillna(method='bfill')

C:\Users\gmpal\AppData\Local\Temp\ipykernel_27328\2870717442.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='bfill')


To measure the perofrmances of a persistent model, we create a new column that repeats the last available time at 45 minutes (previous hour) for the next four steps. This way, prediction for 05:00, 05:15, 05:30, 05:45 will always be 04:45. 

In [16]:
# Function to get the value of Total Load at the previous day, 45 minutes past the hour
def get_previous_day_value(df, current_time):
    previous_day = current_time - pd.DateOffset(hours=1)
    # change minutes to 45
    previous_time = previous_day.replace(minute=45)
    if previous_time in df.index:
        return df.loc[previous_time, 'Total Load']
    return None

# Apply the function to create the 'Total Load Persistence' column
data['Total Load Persistence'] = data.index.to_series().apply(lambda x: get_previous_day_value(data, x))

print(data[['Total Load','Total Load Persistence']].head(10))

                           Total Load  Total Load Persistence
Datetime                                                     
2024-08-23 04:15:00+02:00     7378.27                 7215.68
2024-08-23 04:00:00+02:00     7361.58                 7215.68
2024-08-23 03:45:00+02:00     7215.68                 7331.02
2024-08-23 03:30:00+02:00     7273.88                 7331.02
2024-08-23 03:15:00+02:00     7407.23                 7331.02
2024-08-23 03:00:00+02:00     7386.33                 7331.02
2024-08-23 02:45:00+02:00     7331.02                 7186.31
2024-08-23 02:30:00+02:00     7311.89                 7186.31
2024-08-23 02:15:00+02:00     7233.88                 7186.31
2024-08-23 02:00:00+02:00     7282.41                 7186.31


In [17]:
data = data.dropna()

In [20]:
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error, root_mean_squared_error

starting_hour = data.index.min()
ending_hour = data.index.max()

metrics = pd.DataFrame(columns=['1-step-ahead','2-steps-ahead','3-steps-ahead','4-steps-ahead'], index=['rmse','mse','mape','mae'])

one_step = data.loc[data.index.map(lambda x: x.minute == 00)]
two_steps = data.loc[data.index.map(lambda x: x.minute == 15)]
three_steps = data.loc[data.index.map(lambda x: x.minute == 30)]
four_steps = data.loc[data.index.map(lambda x: x.minute == 45)]

for step, step_data in enumerate([one_step,two_steps,three_steps,four_steps]):

    y_true = step_data['Total Load']
    y_hat = step_data['Total Load Persistence']

    rmse = root_mean_squared_error(y_true, y_hat)
    mse = mean_squared_error(y_true,y_hat)
    mape = mean_absolute_percentage_error(y_true,y_hat)
    mae = mean_absolute_error(y_true,y_hat)

    metrics.iloc[:,step] = [rmse,mse,mape,mae]


In [21]:
metrics

1-step-ahead 2-steps-ahead 3-steps-ahead  4-steps-ahead
rmse    135.196172    226.247432    308.989001     385.832047
mse   18278.004867  51187.900338  95474.202756  148866.368247
mape      0.010833      0.018201      0.024934       0.031287
mae     102.548216    173.040749      237.5281     297.761971